In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

### 1. Iris - sns.load_dataset('Iris')

a. 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width), 꽃받침길이(petal_length), 꽃받침폭(petal_width)의 평균, 표준편차 등 기초통계량(describe())을 구하시오.

b. 3분위수 (Q3)와 1분위수(Q1)의 차이보다 1.5배 크거나 작은 데이터는 이상치이다. 즉, Q1 - 1.5 * (Q3 - Q1)보다 작은 데이터 혹은 Q1 + 1.5 * (Q3 - Q1)보다 큰 데이터는 이상치이므로 제거하고 위의 4가지 항목에 대해서 평균, 표준편차를 구하시오.

In [3]:
import seaborn as sns
iris = sns.load_dataset("Iris")

In [4]:
# a
iris.groupby(iris.species)['sepal_length', 'sepal_width', 'petal_length', 'petal_width'].describe().T

species                setosa  versicolor  virginica
sepal_length count  50.000000   50.000000  50.000000
             mean    5.006000    5.936000   6.588000
             std     0.352490    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
sepal_width  count  50.000000   50.000000  50.000000
             mean    3.428000    2.770000   2.974000
             std     0.379064    0.313798   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
petal_length count  50.000000   50.000000  50.000000
             mean    1.462000    4.260000   5.552000
             std     0.173664    0.469911   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.550000
             75%     1.575000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
petal_width  count  50.000000   50.000000  50.000000
             mean    0.246000    1.326000   2.026000
             std     0.105386    0.197753   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000

In [5]:
def q3cut(x):

    q1 = x.quantile(.25)
    q3 = x.quantile(.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    higher = q3 + 1.5 * iqr
    x[(x > higher) | (x < lower)] = np.nan
    outlier = x.isnull().sum() != 0 
    return outlier, x.mean(), x.std()

In [6]:
species = []
items = []
for i in ['setosa', 'versicolor', 'virginica']:
    for k in ['sepal_length','sepal_width','petal_length','petal_width']:
        lists = iris[iris.species == i][k]
        species.append(i)
        items.append(k)


In [7]:
iris_get_new = pd.DataFrame({
    'species': species,
    'items': items
})
new_iris = iris_get_new.set_index(['species', 'items'])
new_iris

Empty DataFrame
Columns: []
Index: [(setosa, sepal_length), (setosa, sepal_width), (setosa, petal_length), (setosa, petal_width), (versicolor, sepal_length), (versicolor, sepal_width), (versicolor, petal_length), (versicolor, petal_width), (virginica, sepal_length), (virginica, sepal_width), (virginica, petal_length), (virginica, petal_width)]

In [8]:
mean_list = []
std_list = []
outlier_list = []
for i in ['setosa', 'versicolor', 'virginica']:
    for k in ['sepal_length','sepal_width','petal_length','petal_width']:
        lists = iris[iris.species == i][k]
        outlier, mean, std = q3cut(lists)
        outlier_list.append(outlier)
        mean_list.append(mean)
        std_list.append(std)

In [9]:
new_iris['outlier'] = outlier_list
new_iris['mean'] = mean_list
new_iris['std'] = std_list
new_iris

outlier      mean       std
species    items                                    
setosa     sepal_length    False  5.006000  0.352490
           sepal_width      True  3.431250  0.320343
           petal_length     True  1.460870  0.129062
           petal_width      True  0.233333  0.085883
versicolor sepal_length    False  5.936000  0.516171
           sepal_width     False  2.770000  0.313798
           petal_length     True  4.285714  0.437798
           petal_width     False  1.326000  0.197753
virginica  sepal_length     True  6.622449  0.593459
           sepal_width      True  2.955319  0.260275
           petal_length    False  5.552000  0.551895
           petal_width     False  2.026000  0.274650

### 2. Titanic - sns.load_dataset('titanic')

a. 타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 열인 category1 열을 만드시오. category1 카테고리는 다음과 같이 정의됨
    1) 20살이 넘으면 성별을 그대로 사용한다.
    2) 20살 미만이면 성별에 관계없이 "child"라고 한다.

b. 타이타닉호의 승객 중 나이를 명시하지 않는 고객은 나이를 명시한 고객의 평균 나이 값이 되도록 titanic 데이터 프레임을 고치시오.

c. 성별, 선실(class)별, 출발지(embark_town)별 생존율을 구하시오.

d. 타이타닉호 승객을 '미성년자', '청년', '중년', '장년', '노년' 나이 그룹으로 나누고, 각 그룹별 생존율을 구하시오.
    bin = [1, 20, 30, 50, 70, 100]
    labels = ['미성년자', '청년', '중년', '장년', '노년']

e. qcut 명령으로 세 개의 나이 그룹을 만들고, 나이 그룹별 남녀 성비와 생존율을 구하시오.

In [21]:
titanic = sns.load_dataset("titanic")
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [22]:
titanic.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [23]:
# a 
titanic["category1"] = titanic.apply(lambda r: r.sex if r.age >= 20 else "child", axis=1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


In [24]:
# b
titanic['age'] = titanic['age'].fillna(round(titanic.age.mean(), 2))
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


In [25]:
# c 성별 생존
titanic2 = titanic[titanic.survived == 1]
titanic2

titanic3 = titanic2[titanic2.sex == 'female'] # 여자 생존
titanic3

titanic3.count()['survived']

233

In [26]:
titanic4 = titanic2[titanic2.sex == 'male'] # 남자 생존
titanic4

titanic4.count()['survived']

109

In [27]:
# 성별 생존 결과값 (전체 인원 기준으로 생존율 구함)
x = titanic4.count()['survived'] / titanic.count()['survived']
y = titanic3.count()['survived'] / titanic.count()['survived']

print('남성 생존율 :', x)
print('여성 생존율 :', y)

남성 생존율 : 0.122334455667789
여성 생존율 : 0.2615039281705948


In [28]:
# c 선실별 생존
titanic5 = titanic2[titanic2.pclass == 1]
titanic5

titanic6 = titanic2[titanic2.pclass == 2]
titanic6

titanic7 = titanic2[titanic2.pclass == 3]
titanic7

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,female
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False,female
10,1,3,female,4.0,1,1,16.7000,S,Third,child,False,G,Southampton,yes,False,child
19,1,3,female,29.7,0,0,7.2250,C,Third,woman,False,NaN,Cherbourg,yes,True,child
22,1,3,female,15.0,0,0,8.0292,Q,Third,child,False,NaN,Queenstown,yes,True,child
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,1,3,male,32.0,0,0,56.4958,S,Third,man,True,NaN,Southampton,yes,True,male
855,1,3,female,18.0,0,1,9.3500,S,Third,woman,False,NaN,Southampton,yes,False,child
858,1,3,female,24.0,0,3,19.2583,C,Third,woman,False,NaN,Cherbourg,yes,False,female
869,1,3,male,4.0,1,1,11.1333,S,Third,child,False,NaN,Southampton,yes,False,child


In [29]:
# 선실별 생존 결과값 (전체 인원 기준으로 생존율 구함)
a = titanic5.count()['survived'] / titanic.count()['survived']
b = titanic6.count()['survived'] / titanic.count()['survived']
c = titanic7.count()['survived'] / titanic.count()['survived']

print('First Class 생존율 :', a)
print('Second Class 생존율 :', b)
print('Third Class 생존율 :', c)

First Class 생존율 : 0.1526374859708193
Second Class 생존율 : 0.09764309764309764
Third Class 생존율 : 0.1335578002244669


In [30]:
# c 출발지 생존
titanic8 = titanic2[titanic2.embarked == 'S']
titanic8

titanic9 = titanic2[titanic2.embarked == 'C']
titanic9

titanic10 = titanic2[titanic2.embarked == 'Q']
titanic10

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
22,1,3,female,15.0,0,0,8.0292,Q,Third,child,False,NaN,Queenstown,yes,True,child
28,1,3,female,29.7,0,0,7.8792,Q,Third,woman,False,NaN,Queenstown,yes,True,child
32,1,3,female,29.7,0,0,7.7500,Q,Third,woman,False,NaN,Queenstown,yes,True,child
44,1,3,female,19.0,0,0,7.8792,Q,Third,woman,False,NaN,Queenstown,yes,True,child
47,1,3,female,29.7,0,0,7.7500,Q,Third,woman,False,NaN,Queenstown,yes,True,child
82,1,3,female,29.7,0,0,7.7875,Q,Third,woman,False,NaN,Queenstown,yes,True,child
109,1,3,female,29.7,1,0,24.1500,Q,Third,woman,False,NaN,Queenstown,yes,False,child
156,1,3,female,16.0,0,0,7.7333,Q,Third,woman,False,NaN,Queenstown,yes,True,child
186,1,3,female,29.7,1,0,15.5000,Q,Third,woman,False,NaN,Queenstown,yes,False,child
198,1,3,female,29.7,0,0,7.7500,Q,Third,woman,False,NaN,Queenstown,yes,True,child


In [31]:
# 출발지별 생존 결과값 (전체 인원 기준으로 생존율 구함)
i = titanic8.count()['survived'] / titanic.count()['survived']
o = titanic9.count()['survived'] / titanic.count()['survived']
q = titanic10.count()['survived'] / titanic.count()['survived']

print('S출발지 생존욜 :', i)
print('C출발지 생존율 :', o)
print('Q출발지 생존율 :', q)

S출발지 생존욜 : 0.24354657687991021
C출발지 생존율 : 0.10437710437710437
Q출발지 생존율 : 0.03367003367003367


In [32]:
# d

bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "중년", "장년", "노년"]
titanics = pd.cut(titanic['age'], bins, labels=labels)

print(titanics)

0        청년
1        중년
2        청년
3        중년
4        중년
       ... 
886      청년
887    미성년자
888      청년
889      청년
890      중년
Name: age, Length: 891, dtype: category
Categories (5, object): [미성년자 < 청년 < 중년 < 장년 < 노년]


In [33]:
# e. qcut 명령으로 세 개의 나이 그룹을 만들고, 나이 그룹별 남녀 성비와 생존율을 구하시오.

def q3cut(s):
    return pd.qcut(s, 3, labels=["젊음", "중간", "늙음"]).astype(str)


titanic["age_class"] = titanic.groupby(titanic.sex).age.transform(q3cut)
titanic[["age", "age_class", "sex"]].tail(10)

,age,age_class,sex
881,33.0,늙음,male
882,22.0,젊음,female
883,28.0,중간,male
884,25.0,젊음,male
885,39.0,늙음,female
886,27.0,젊음,male
887,19.0,젊음,female
888,29.7,중간,female
889,26.0,젊음,male
890,32.0,늙음,male


In [34]:
titanic_class = titanic[titanic.survived == 1]

titanic_class_w = titanic_class[titanic.sex == 'female']
titanic_class_m = titanic_class[titanic.sex == 'male']

In [35]:
titanic11 = titanic_class_w[titanic.age_class == '젊음']
titanic12 = titanic_class_w[titanic.age_class == '중간']
titanic13 = titanic_class_w[titanic.age_class == '늙음']

titanic14 = titanic_class_m[titanic.age_class == '젊음']
titanic15 = titanic_class_m[titanic.age_class == '중간']
titanic16 = titanic_class_m[titanic.age_class == '늙음']

In [36]:
female_young = titanic11.count()['age_class'] / titanic3.count()['survived']
female = titanic12.count()['age_class'] / titanic3.count()['survived']
female_old = titanic13.count()['age_class'] / titanic3.count()['survived']

male_young = titanic14.count()['age_class'] / titanic4.count()['survived']
male = titanic15.count()['age_class'] / titanic4.count()['survived']
male_old = titanic16.count()['age_class'] / titanic4.count()['survived']

In [37]:
# 나이 그룹별 남녀 생존율 (생존한 사람 중 남녀 비율)
w_young = titanic11.count()['age_class']/(titanic11.count()['age_class']+titanic14.count()['age_class'])
m_young = titanic14.count()['age_class']/(titanic11.count()['age_class']+titanic14.count()['age_class'])

w = titanic12.count()['age_class']/(titanic12.count()['age_class']+titanic15.count()['age_class'])
m = titanic15.count()['age_class']/(titanic12.count()['age_class']+titanic15.count()['age_class'])

w_old = titanic13.count()['age_class']/(titanic13.count()['age_class']+titanic16.count()['age_class'])
m_old = titanic16.count()['age_class']/(titanic13.count()['age_class']+titanic16.count()['age_class'])

print('젊은여자 생존율 : 젊은남자 생존율 = ', w_young, ':', m_young)
print('중간여자 생존율 : 중간남자 생존율 = ', w, ':', m)
print('늙은여자 생존율 : 늙은남자 생존율 = ', w_old, ':', m_old)

젊은여자 생존율 : 젊은남자 생존율 =  0.648854961832061 : 0.3511450381679389
중간여자 생존율 : 중간남자 생존율 =  0.7222222222222222 : 0.2777777777777778
늙은여자 생존율 : 늙은남자 생존율 =  0.6859504132231405 : 0.3140495867768595


In [38]:
# 남녀 성비 생존율 (생존한 사람 중 나이별 비율)

print('젊은 여성 생존율 :', female_young)
print('중간 여성 생존율 :', female)
print('늙은 여성 생존율 :', female_old)
print('젊은 남성 생존율 :', male_young)
print('중간 남성 생존율 :', male)
print('늙은 남성 생존율 :', male_old)


젊은 여성 생존율 : 0.3648068669527897
중간 여성 생존율 : 0.27896995708154504
늙은 여성 생존율 : 0.3562231759656652
젊은 남성 생존율 : 0.42201834862385323
중간 남성 생존율 : 0.22935779816513763
늙은 남성 생존율 : 0.3486238532110092


### 3. Mile Per Gallon - sns.load_dataset('mpg')

a. 배기량(displacement) 대비 마력(horsepower) 열(hp_per_cc)을 추가하시오.

b. name으로부터 manufacturer(제조사)와 모델을 추출하여 새로운 열 manufacturer와 model을 추가하고, name 열은 삭제하시오.

c. 엔진의 실린더(cylinders) 갯수별 연비(mpg)의 평균을 구하시오.

d. 생산지(origin)별 배기량 대비 마력(hp_per_cc)의 평균을 구하시오.

e. 모델이 5개 이상인 제조사에 대하여 연비(mpg)의 평균이 가장 좋은 제조사 Top5를 구하시오.

In [39]:
mpg = sns.load_dataset("mpg")
mpg

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger


In [40]:
mpg.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
count,398.000000,398.000000,398.000000,392.000000,398.000000,398.000000,398.000000
mean,23.514573,5.454774,193.425879,104.469388,2970.424623,15.568090,76.010050
std,7.815984,1.701004,104.269838,38.491160,846.841774,2.757689,3.697627
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000
25%,17.500000,4.000000,104.250000,75.000000,2223.750000,13.825000,73.000000
50%,23.000000,4.000000,148.500000,93.500000,2803.500000,15.500000,76.000000
75%,29.000000,8.000000,262.000000,126.000000,3608.000000,17.175000,79.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000


In [41]:
# a. 배기량(displacement) 대비 마력(horsepower) 열(hp_per_cc)을 추가하시오.
mpg['hp_per_cc'] = mpg['displacement'] / mpg['horsepower']
mpg.tail()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl,1.627907
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup,1.865385
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage,1.607143
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger,1.518987
397,31.0,4,119.0,82.0,2720,19.4,82,usa,chevy s-10,1.451220


In [42]:
# b. name으로부터 manufacturer(제조사)와 모델을 추출하여 새로운 열 manufacturer와 model을 추가하고, name 열은 삭제하시오.

def get_manufacturer(s):
    return s.split()[0]

def get_model(s):
    return ' '.join(s.split()[1:])

# 스트링으로 바로 dataframe에 넣어줄 경우 apply를 적용하고, 리스트를 넣어주는 경우는 mpg['인덱스'] = 리스트 이름
mpg['manufacturer'] = mpg.name.apply(get_manufacturer)
mpg['model'] = mpg.name.apply(get_model)
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc,manufacturer,model
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,2.361538,chevrolet,chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,2.121212,buick,skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,2.120000,plymouth,satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,2.026667,amc,rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,2.157143,ford,torino


In [43]:
# c. 엔진의 실린더(cylinders) 갯수별 연비(mpg)의 평균을 구하시오.

mpg['mpg_per_cylinders'] = mpg['mpg'] / mpg['cylinders']
mpg.tail()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,hp_per_cc,manufacturer,model,mpg_per_cylinders
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl,1.627907,ford,mustang gl,6.75
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup,1.865385,vw,pickup,11.00
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage,1.607143,dodge,rampage,8.00
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger,1.518987,ford,ranger,7.00
397,31.0,4,119.0,82.0,2720,19.4,82,usa,chevy s-10,1.451220,chevy,s-10,7.75


In [44]:
del mpg['name']

In [45]:
mpg

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manufacturer,model,mpg_per_cylinders
0,18.0,8,307.0,130.0,3504,12.0,70,usa,2.361538,chevrolet,chevelle malibu,2.250
1,15.0,8,350.0,165.0,3693,11.5,70,usa,2.121212,buick,skylark 320,1.875
2,18.0,8,318.0,150.0,3436,11.0,70,usa,2.120000,plymouth,satellite,2.250
3,16.0,8,304.0,150.0,3433,12.0,70,usa,2.026667,amc,rebel sst,2.000
4,17.0,8,302.0,140.0,3449,10.5,70,usa,2.157143,ford,torino,2.125
...,...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,1.627907,ford,mustang gl,6.750
394,44.0,4,97.0,52.0,2130,24.6,82,europe,1.865385,vw,pickup,11.000
395,32.0,4,135.0,84.0,2295,11.6,82,usa,1.607143,dodge,rampage,8.000
396,28.0,4,120.0,79.0,2625,18.6,82,usa,1.518987,ford,ranger,7.000


In [46]:
# d. 생산지(origin)별 배기량 대비 마력(hp_per_cc)의 평균을 구하시오.

mpg_usa = mpg[mpg.origin == 'usa']
mpg_europe = mpg[mpg.origin == 'europe']
mpg_japan = mpg[mpg.origin == 'japan']

In [47]:
print('미국 배기량 대비 마력 평균 :', mpg_usa.hp_per_cc.sum() / mpg_usa.hp_per_cc.count())
print('일본 배기량 대비 마력 평균 :', mpg_japan.hp_per_cc.sum() / mpg_japan.hp_per_cc.count())
print('유럽 배기량 대비 마력 평균 :', mpg_europe.hp_per_cc.sum() / mpg_europe.hp_per_cc.count())

미국 배기량 대비 마력 평균 : 2.0482890171294357
일본 배기량 대비 마력 평균 : 1.3041195699620984
유럽 배기량 대비 마력 평균 : 1.405596991707455


In [48]:
# e

mpg_lists = mpg.manufacturer.value_counts()
allowed = mpg_lists[mpg_lists >= 5].index.tolist()

In [49]:
mpg[mpg['manufacturer'].isin(allowed)]

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,hp_per_cc,manufacturer,model,mpg_per_cylinders
0,18.0,8,307.0,130.0,3504,12.0,70,usa,2.361538,chevrolet,chevelle malibu,2.250
1,15.0,8,350.0,165.0,3693,11.5,70,usa,2.121212,buick,skylark 320,1.875
2,18.0,8,318.0,150.0,3436,11.0,70,usa,2.120000,plymouth,satellite,2.250
3,16.0,8,304.0,150.0,3433,12.0,70,usa,2.026667,amc,rebel sst,2.000
4,17.0,8,302.0,140.0,3449,10.5,70,usa,2.157143,ford,torino,2.125
...,...,...,...,...,...,...,...,...,...,...,...,...
392,27.0,4,151.0,90.0,2950,17.3,82,usa,1.677778,chevrolet,camaro,6.750
393,27.0,4,140.0,86.0,2790,15.6,82,usa,1.627907,ford,mustang gl,6.750
394,44.0,4,97.0,52.0,2130,24.6,82,europe,1.865385,vw,pickup,11.000
395,32.0,4,135.0,84.0,2295,11.6,82,usa,1.607143,dodge,rampage,8.000


In [50]:
mpg[mpg['manufacturer'].isin(allowed)][['manufacturer','mpg']].groupby('manufacturer').mean()

,mpg
manufacturer,
amc,18.246429
audi,26.714286
buick,19.182353
chevrolet,20.472093
chrysler,17.266667
datsun,31.113043
dodge,22.060714
fiat,28.912500
ford,19.694118


In [51]:
mpg[mpg['manufacturer'].isin(allowed)].pivot_table('mpg','manufacturer')

,mpg
manufacturer,
amc,18.246429
audi,26.714286
buick,19.182353
chevrolet,20.472093
chrysler,17.266667
datsun,31.113043
dodge,22.060714
fiat,28.912500
ford,19.694118


In [52]:
mpg[mpg['manufacturer'].isin(allowed)].pivot_table('mpg','manufacturer').sort_values(by='mpg', ascending=False)[:5]

,mpg
manufacturer,
vw,39.016667
honda,33.761538
renault,32.880000
datsun,31.113043
mazda,30.860000
